## Download Imagenette, a subset of ImageNet

In [1]:
from fastai.vision.all import *
dataset = untar_data(URLs.IMAGENETTE)

## Initiate a fastai DataBlock and define data preprocessing/augmentation

In [2]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                        get_items=get_image_files,
                        get_y=parent_label,
                        item_tfms=Resize(460),
                        batch_tfms=aug_transforms(size=244, min_scale=0.75))
dls = dblock.dataloaders(dataset, bs=32)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


## Get a baseline model using dynamically changing learning rate

In [3]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.606717,4.534217,0.264750,03:45
1,1.216525,1.448804,0.551158,03:17
2,0.976563,2.245200,0.530620,03:28
3,0.722283,0.657878,0.790142,03:19
4,0.605014,0.555290,0.828603,03:21


## What's the mean and standard deviation of our data?

In [6]:
x, y = dls.one_batch()
x.mean(dim=[0, 2, 3]) # skip the alpha channel

TensorImage([0.4329, 0.4313, 0.4141], device='cuda:0')

In [7]:
x.std(dim=[0, 2, 3]) # skip the alpha channel

TensorImage([0.2771, 0.2656, 0.2884], device='cuda:0')

## Let's standarize the data

In [10]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                        get_items=get_image_files,
                        get_y=parent_label,
                        item_tfms=Resize(460),
                        batch_tfms=[*aug_transforms(size=244, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])   # fastai has predefined stats for ImageNet
dls = dblock.dataloaders(dataset, bs=32)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [11]:
x, y = dls.one_batch()
x.mean(dim=[0, 2, 3]) # skip the alpha channel

TensorImage([-0.1948, -0.0827,  0.1260], device='cuda:0')

In [12]:
x.std(dim=[0, 2, 3]) # skip the alpha channel

TensorImage([1.2606, 1.2181, 1.3130], device='cuda:0')

## Does it improve our accuracy?

In [13]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.608027,2.034141,0.408140,03:22
1,1.213673,1.150071,0.638163,03:20
2,0.938707,1.272448,0.616131,03:19
3,0.732879,0.652468,0.795743,03:19
4,0.621922,0.578882,0.818895,03:18


## Does progressive resizing improve training time or/and accuracy?

In [3]:
# smaller images
dblock_small = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                        get_items=get_image_files,
                        get_y=parent_label,
                        item_tfms=Resize(460),
                        batch_tfms=[*aug_transforms(size=128, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])   # fastai has predefined stats for ImageNet
dls_small = dblock_small.dataloaders(dataset, bs=32)

# regularly sized images
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                        get_items=get_image_files,
                        get_y=parent_label,
                        item_tfms=Resize(460),
                        batch_tfms=[*aug_transforms(size=244, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])   # fastai has predefined stats for ImageNet
dls = dblock.dataloaders(dataset, bs=32)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck
Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


## Train the model for 4 epochs with 128x128 images

In [5]:
model = xresnet50()
learn = Learner(dls_small, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.600482,1.979397,0.482450,02:28
1,1.167944,1.295597,0.608290,02:27
2,0.837772,0.750104,0.765123,02:24
3,0.650627,0.595431,0.810680,02:25


## Fine tune it with 244x244 images

In [6]:
learn.dls = dls
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.763961,0.815504,0.750560,03:15


epoch,train_loss,valid_loss,accuracy,time
0,0.611027,0.526435,0.836819,03:27
1,0.582712,0.614127,0.814040,03:22
2,0.520653,0.516086,0.843167,03:17
3,0.494449,0.416041,0.874907,03:22
4,0.419290,0.406381,0.880881,03:21
